In [1]:
import pandas as pd


ROOT_DATA = "./data/raw"

In [3]:
df_acc = pd.read_csv(f"{ROOT_DATA}/accidente-de-trafico-en-bogota-entre-2007-y-2017-geopoint.csv", sep=';')
df_vm = pd.read_csv(f"{ROOT_DATA}/vm_acc_via.csv", sep=';')

print(f"Datos totales en Accidente de Tráfico: {len(df_acc)}")
print(f"Datos totales en vm_acc_via: {len(df_vm)}")

Datos totales en Accidente de Tráfico: 330063
Datos totales en vm_acc_via: 396244


## Count duplicated values

In [17]:
# Duplicados en código de accidente
duplicados_acc = df_acc['CODIGO_ACCIDENTE'].duplicated().sum()
print(f"Duplicados de CODIGO_ACCIDENTE en df_acc: {duplicados_acc}")

duplicados_vm = df_vm['CODIGO_ACCIDENTE'].duplicated().sum()
print(f"Duplicados de CODIGO_ACCIDENTE en df_vm: {duplicados_vm}")

Duplicados de CODIGO_ACCIDENTE en df_acc: 0
Duplicados de CODIGO_ACCIDENTE en df_vm: 75005


In [18]:
# Duplicados en formulario
duplicados_acc_for = df_acc['FORMULARIO'].duplicated().sum()
print(f"Duplicados de FORMULARIO en df_acc: {duplicados_acc_for}")

duplicados_vm_for = df_vm['FORMULARIO'].duplicated().sum()
print(f"Duplicados de FORMULARIO en df_vm: {duplicados_vm_for}")

Duplicados de FORMULARIO en df_acc: 0
Duplicados de FORMULARIO en df_vm: 75005


In [19]:
# Duplicados de OBJECTID
duplicados_vm_obj = df_vm['OBJECTID'].duplicated().sum()
print(f"Duplicados de OBJECTID en df_vm: {duplicados_vm_obj}")

Duplicados de OBJECTID en df_vm: 0


In [20]:
# Duplicados en df_vm
duplicates_vm = df_vm.duplicated(subset=['FORMULARIO','CODIGO_ACCIDENTE'], keep=False)
print(f"Cantidad de duplicados de FORMULARIO y CODIGO_ACCIDENTE en df_vm: {duplicados_vm.sum()}")

Cantidad de duplicados de FORMULARIO y CODIGO_ACCIDENTE en df_vm: 75005


In [21]:
# Joins
df_merged_objectID = pd.merge(df_acc, df_vm, on='OBJECTID', how='inner')
df_merged_formulario = pd.merge(df_acc, df_vm, on='FORMULARIO', how='inner')
df_merged_cod_accidente = pd.merge(df_acc, df_vm, on='CODIGO_ACCIDENTE', how='inner')

print(f"Número de registros en el DataFrame resultante unido por objectID: {len(df_merged_objectID)}")
print(f"Número de registros en el DataFrame resultante unido por formulario: {len(df_merged_formulario)}")
print(f"Número de registros en el DataFrame resultante unido por codigo_accidente: {len(df_merged_cod_accidente)}")


Número de registros en el DataFrame resultante unido por objectID: 179310
Número de registros en el DataFrame resultante unido por formulario: 396244
Número de registros en el DataFrame resultante unido por codigo_accidente: 396244


In [22]:
print(f"Accidentes con datos de vía: {df_merged_cod_accidente['CODIGO_VIA'].notna().sum()}")
print(f"Accidentes sin datos de vía: {df_merged_cod_accidente['CODIGO_VIA'].isna().sum()}")
df_merged_cod_accidente.columns

Accidentes con datos de vía: 396244
Accidentes sin datos de vía: 0


Index(['Geo Point', 'OBJECTID_x', 'FORMULARIO_x', 'CODIGO_ACCIDENTE',
       'FECHA_OCURRENCIA', 'HORA_OCURRENCIA', 'ANO_OCURRENCIA',
       'MES_OCURRENCIA', 'DIA_OCURRENCIA', 'DIRECCION', 'GRAVEDAD', 'CLASE',
       'LOCALIDAD', 'AREAS_TRANSITO', 'CUADRANTE_TRANSITO', 'MUNICIPIO',
       'TEXTO_OBSERVACIONES', 'FECHA_HORA', 'OBJECTID_y', 'FORMULARIO_y',
       'CODIGO_VIA', 'GEOMETRICA_A', 'GEOMETRICA_B', 'GEOMETRICA_C',
       'UTILIZACION', 'CALZADAS', 'CARRILES', 'MATERIAL', 'ESTADO',
       'CONDICIONES', 'ILUMINACION_A', 'ILUMINACION_B', 'AGENTE_TRANSITO',
       'SEMAFORO', 'VISUAL', 'CODIGO'],
      dtype='object')

In [23]:
# Filas donde FORMULARIO_x y FORMULARIO_y son diferentes (y FORMULARIO_y no es nulo)
diff_formularios = df_merged_cod_accidente[df_merged_cod_accidente['FORMULARIO_x'] != df_merged_cod_accidente['FORMULARIO_y']]

print(f"Cantidad de registros donde FORMULARIO difiere: {len(diff_formularios)}")

Cantidad de registros donde FORMULARIO difiere: 0


In [24]:
# Eliminar una columna de formularios
df_merged_cod_accidente.drop('FORMULARIO_y', axis=1, inplace=True)
df_merged_cod_accidente.rename(columns={'FORMULARIO_x': 'FORMULARIO'}, inplace=True)

df_merged_cod_accidente.columns

Index(['Geo Point', 'OBJECTID_x', 'FORMULARIO', 'CODIGO_ACCIDENTE',
       'FECHA_OCURRENCIA', 'HORA_OCURRENCIA', 'ANO_OCURRENCIA',
       'MES_OCURRENCIA', 'DIA_OCURRENCIA', 'DIRECCION', 'GRAVEDAD', 'CLASE',
       'LOCALIDAD', 'AREAS_TRANSITO', 'CUADRANTE_TRANSITO', 'MUNICIPIO',
       'TEXTO_OBSERVACIONES', 'FECHA_HORA', 'OBJECTID_y', 'CODIGO_VIA',
       'GEOMETRICA_A', 'GEOMETRICA_B', 'GEOMETRICA_C', 'UTILIZACION',
       'CALZADAS', 'CARRILES', 'MATERIAL', 'ESTADO', 'CONDICIONES',
       'ILUMINACION_A', 'ILUMINACION_B', 'AGENTE_TRANSITO', 'SEMAFORO',
       'VISUAL', 'CODIGO'],
      dtype='object')

## Save dataset

In [8]:
df = (
    pd.merge(df_acc, df_vm, on="OBJECTID", how="left")
    .drop(columns=["FORMULARIO_y", "CODIGO_ACCIDENTE_y"])
    .rename(
        columns={"FORMULARIO_x": "FORMULARIO", "CODIGO_ACCIDENTE_x": "CODIGO_ACCIDENTE"}
    )
)

# Parse geo points
geopoints = df["Geo Point"].str.split(",", expand=True).astype(float)
geopoints.columns = ["latitude", "longitude"]

# # Optional: drop original "Geo Point" column
df.drop(columns=["Geo Point"], inplace=True)

df.to_csv("./data/processed/dataset.csv", index=False)

## Read processed dataset

In [9]:
pd.read_csv("./data/processed/dataset.csv")

,OBJECTID,FORMULARIO,CODIGO_ACCIDENTE,FECHA_OCURRENCIA,HORA_OCURRENCIA,ANO_OCURRENCIA,MES_OCURRENCIA,DIA_OCURRENCIA,DIRECCION,GRAVEDAD,...,CARRILES,MATERIAL,ESTADO,CONDICIONES,ILUMINACION_A,ILUMINACION_B,AGENTE_TRANSITO,SEMAFORO,VISUAL,CODIGO
0,646545,A00410124,306211,Fri Sep 05 00:00:00 CEST 2008,12:50:00,2008,SEPTIEMBRE,VIERNES,AK 68-44 02,CON HERIDOS,...,DOS,CONCRETO,BUENO,SECA,SIN,BUENA,NaN,NaN,NaN,646545-298258-1
1,563782,839933000,291565,Tue May 20 00:00:00 CEST 2008,23:40:00,2008,MAYO,MARTES,AV CIUDAD DE CALI-CL 66 02,SOLO DANOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4398952,A000038718,4398952,Thu Dec 04 00:00:00 CET 2014,17:30:00,2014,DICIEMBRE,JUEVES,AV AVENIDA CIUDAD DE CALI-CL 66A 02,SOLO DANOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1612,726750500,30243,Fri Jun 29 00:00:00 CEST 2007,19:30:00,2007,JUNIO,VIERNES,TR 40A-5 32,CON HERIDOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,569692,840564600,297475,Mon Jul 21 00:00:00 CEST 2008,07:30:00,2008,JULIO,LUNES,KR 68-22A 02,SOLO DANOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330058,1707169,A00719849,352633,Thu Mar 04 00:00:00 CET 2010,14:30:00,2010,MARZO,JUEVES,KR 99-CL21BIS 02,SOLO DANOS,...,CUATRO O MAS,ASFALTO,BUENO,SECA,SIN,NaN,NO,NaN,NaN,1707169-344258-1
330059,9682,712264000,16686,Wed Mar 07 00:00:00 CET 2007,11:00:00,2007,MARZO,MIERCOLES,CL 70-19D S 61,SOLO DANOS,...,TRES O MAS,ASFALTO,BUENO,SECA,SIN,NaN,NO,OPERANDO,NaN,9682-4400930-1
330060,4421360,A12506,4421360,Wed Aug 19 00:00:00 CEST 2015,16:50:00,2015,AGOSTO,MIERCOLES,AV AVENIDA DEL SUR-CL 67 S 02,SOLO DANOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330061,60219,828721300,673,Sat Jan 05 00:00:00 CET 2008,19:27:00,2008,ENERO,SABADO,KR 50-55 2,SOLO DANOS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
